# Importing Libraries

In [20]:
import pandas as pd
from utils import common_functions 
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_error
import mlflow
from mlflow.exceptions import RestException
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.backend import clear_session


# Iniciando funciones

In [2]:
create_sequences = common_functions().create_sequences

# Leyendo data y limpiandola

In [3]:
path_data = f'../sandbox/pepe.csv'
df = pd.read_csv(path_data,header=0)

In [128]:
#Limpieando la data 

# Convertir "Open time" y "Close time" a datetime y darle formato ISO 8601
df['Open time_date'] = pd.to_datetime(df['Open time'], unit='ms').apply(lambda x: x.isoformat())
df['Close time_date'] = pd.to_datetime(df['Close time'], unit='ms').apply(lambda x: x.isoformat())

#Escalando el valor de el campo Close
df['closex1M'] = df['Close'] * 1000000

# NORMALIZAR LA DATA QUITAR ESA TRASFORMACION

scaler = MinMaxScaler(feature_range=(0, 1))  # O feature_range=(-1, 1)
df['Close_Norm'] = scaler.fit_transform(df['closex1M'].to_numpy().reshape(-1, 1))


In [129]:
#seleccionando los campos que necesitamos
select = ['Close time_date','closex1M']
df_clean = df[select]

In [130]:
df_clean.head()

,Close time_date,closex1M
0,2023-05-05T18:59:59.999000,3.07
1,2023-05-05T19:59:59.999000,3.35
2,2023-05-05T20:59:59.999000,3.52
3,2023-05-05T21:59:59.999000,3.20
4,2023-05-05T22:59:59.999000,3.33


# Conexion a Mlflow server
tambien se crean las ejecuciones necesarias


In [131]:
remote_server_uri = "http://34.58.215.162:8080/"  # set to your server URI
mlflow.set_tracking_uri(remote_server_uri)

#Creacion o identificacion del experimento 
try:
    experiment_name = 'SERIET_PLUS_MODELADO' #Puede ser cualquiera siempre y cuando no se troque con otro
    experiment_id = (mlflow
                        .create_experiment(name=experiment_name
                                            ,tags={'created_by':'Juan Carlos Cabrera'})) #importante poner el nombre de quien lo crea
except RestException as r:
    print(r)
    experiment = mlflow.get_experiment_by_name(experiment_name)
    print('Full name',experiment.name)
    experiment_id = experiment.experiment_id

RESOURCE_ALREADY_EXISTS: Experiment 'SERIET_PLUS_MODELADO' already exists.
Full name SERIET_PLUS_MODELADO


## Creando la ejecucion

In [132]:
with mlflow.start_run(experiment_id=experiment_id
                      #,run_name='LSTM'# El nombre se coloca solo la primera vez
                      ,run_id='e0b18f5efa104f5184b5df9a61728da1'
                      ) as run_parent:
    with mlflow.start_run(experiment_id=experiment_id
                          ,run_name='SERIE TEMPORAL SIN PREPROCESADO' # Acá se ponne el nombre de la ejecución
                          ,nested=True) as run_child:
        print('Run creada para SERIE TEMPORAL SIN PREPROCESADO con run_id',run_child.info.run_id)

Run creada para SERIE TEMPORAL SIN PREPROCESADO con run_id 3299b2438f124b60adafbb571cb76db2
🏃 View run SERIE TEMPORAL SIN PREPROCESADO at: http://34.58.215.162:8080/#/experiments/509966702496361241/runs/3299b2438f124b60adafbb571cb76db2
🧪 View experiment at: http://34.58.215.162:8080/#/experiments/509966702496361241
🏃 View run LSTM at: http://34.58.215.162:8080/#/experiments/509966702496361241/runs/e0b18f5efa104f5184b5df9a61728da1
🧪 View experiment at: http://34.58.215.162:8080/#/experiments/509966702496361241


In [10]:
# Ejemplo
#mlflow.log_artifact(wave_img_path,run_id=run_child.info.run_id,artifact_path='resultados')

# Entrenamiento

## Preparacion de la data

In [133]:
df_train = df_clean[['closex1M']].copy()

window_size = 24
X, Y = create_sequences(df_train,window_size=window_size,target_col='closex1M')

X = X.reshape(X.shape[0],X.shape[1])

Y = Y.reshape(-1,1)

#Espliteando la data
X_train, X_vt, y_train, y_vt = train_test_split(X, Y, test_size=0.3, shuffle=False)

x_val, x_test, y_val, y_test = train_test_split(X_vt,y_vt,test_size=0.1, shuffle=False)


# Reshape para que sea compatible con LSTM (samples, time steps, features)
#X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
#X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))


## Creando modelo

In [134]:
capas = [('LSTM',100),('LSTM',50),('LSTM',25),('DENSE',1)]
fn_activation_by_layer = ['relu','relu','relu','relu']
epochs = 10
batch_size = 16
loss_metric = 'mse'
model_metrics = ['mae']
model_result_path = 'model_results.html'
optimizer = 'adam'

In [135]:
# Limpiamos el grafo computacional
clear_session()

model = Sequential([
    Input(shape=(window_size,1)),
    LSTM(100,activation=fn_activation_by_layer[0],return_sequences=True),
    LSTM(50,activation=fn_activation_by_layer[1],return_sequences=True),
    LSTM(25,activation=fn_activation_by_layer[2],return_sequences=False),
    Dense(1,activation=fn_activation_by_layer[3])
])
model.compile(optimizer=optimizer,loss=loss_metric,metrics=model_metrics)

#Entrenando el modelo
history = model.fit(X_train,
                    y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(x_val, y_val),
                    verbose=1
                    )


Epoch 1/10
671/671 ━━━━━━━━━━━━━━━━━━━━ 24s 26ms/step - loss: 1.3262 - mae: 0.5512 - val_loss: 0.6497 - val_mae: 0.4907
Epoch 2/10
671/671 ━━━━━━━━━━━━━━━━━━━━ 20s 25ms/step - loss: 0.0604 - mae: 0.1392 - val_loss: 0.2261 - val_mae: 0.3274
Epoch 3/10
671/671 ━━━━━━━━━━━━━━━━━━━━ 17s 26ms/step - loss: 0.0491 - mae: 0.1269 - val_loss: 0.2912 - val_mae: 0.3659
Epoch 4/10
671/671 ━━━━━━━━━━━━━━━━━━━━ 17s 25ms/step - loss: 0.0293 - mae: 0.0990 - val_loss: 0.1440 - val_mae: 0.2350
Epoch 5/10
671/671 ━━━━━━━━━━━━━━━━━━━━ 17s 25ms/step - loss: 0.0205 - mae: 0.0838 - val_loss: 0.1613 - val_mae: 0.3062
Epoch 6/10
671/671 ━━━━━━━━━━━━━━━━━━━━ 17s 26ms/step - loss: 0.0224 - mae: 0.0881 - val_loss: 0.2640 - val_mae: 0.2888
Epoch 7/10
671/671 ━━━━━━━━━━━━━━━━━━━━ 17s 26ms/step - loss: 0.0252 - mae: 0.0923 - val_loss: 0.3450 - val_mae: 0.4399
Epoch 8/10
671/671 ━━━━━━━━━━━━━━━━━━━━ 17s 26ms/step - loss: 0.0175 - mae: 0.0780 - val_loss: 1.4885 - val_mae: 1.0652
Epoch 9/10
671/671 ━━━━━━━━━━━━━━━━━━━━ 

In [136]:
# Guardando los parametros en la ejecucion 
parameters = {'topology': capas
              ,'data_standarizada':False
              ,'puntos_entrenamiento':X_train.shape[0]
              ,'puntos_validacion':x_val.shape[0]
              ,'puntos_testeo':x_test.shape[0]
              ,'fn_activacion_por_capa':fn_activation_by_layer
              ,'window_size':window_size
              ,'loss_metrics':loss_metric
              ,'model_metrics':model_metrics
              ,'epochs':epochs
              ,'batch_size':batch_size
              ,'optimizer':optimizer}

mlflow.log_params(params=parameters
                  ,run_id=run_child.info.run_id)

In [137]:
#Obteniendo las prediccciones del modelo
train_pred = model.predict(X_train).reshape(-1)
val_pred = model.predict(x_val).reshape(-1)
test_pred = model.predict(x_test).reshape(-1)

336/336 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step
130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step


In [89]:
df_clean.Close_Norm = scaler.inverse_transform(df_clean.Close_Norm.to_numpy().reshape(-1,1))

C:\Users\mefis\AppData\Local\Temp\ipykernel_7864\2942264659.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [121]:
train_pred = scaler.inverse_transform(train_pred.reshape(-1,1)).reshape(-1)
val_pred = scaler.inverse_transform(val_pred.reshape(-1,1)).reshape(-1)
test_pred = scaler.inverse_transform(test_pred.reshape(-1,1)).reshape(-1)

In [139]:
train_test = {'train':train_pred,'test':test_pred}

fig = go.Figure()

dates = df_clean['Close time_date']

date_train = dates[25:len(train_pred)]
date_val = dates[len(train_pred)+25:len(train_pred)+len(val_pred)+25]
date_test = dates[len(train_pred)+len(val_pred)+25:]


fig.add_trace(go.Scatter(x=dates
                            ,y=df_clean.closex1M
                            ,mode='lines'
                            ,name='real'
                            ,line = dict(color='green')))

fig.add_trace(go.Scatter(x=date_train
                            ,y=train_pred
                            ,mode='lines'
                            ,name='train'   
                            ,line=dict(color='blue')))

fig.add_trace(go.Scatter(x=date_val
                            ,y=val_pred
                            ,mode='lines'
                            ,name='val'
                            ,line=dict(color='magenta')))

fig.add_trace(go.Scatter(x=date_test
                            ,y=test_pred
                            ,mode='lines'
                            ,name='test'
                            ,line= dict(color='red')))


fig.write_html(model_result_path)
    
fig.show()



In [140]:
#Guardando la imagen del resultado del modelo
mlflow.log_artifact(model_result_path,run_id=run_child.info.run_id,artifact_path='resultados')

# Guardando metricas

In [141]:

mae_test = mean_absolute_error(y_true=y_test, y_pred=test_pred)
mae_train = mean_absolute_error(y_true=y_train, y_pred=train_pred)
mae_val = mean_absolute_error(y_true=y_val, y_pred=val_pred)


print(f"MAE train: {mae_train},   MAE val: {mae_val},  MAE test: {mae_test}")

metrics = {'mae_train':mae_train,'mae_val':mae_val,'mae_test':mae_test}

MAE train: 0.07801761497591495,   MAE val: 0.22495412451987398,  MAE test: 0.34330152859895136


In [142]:
mlflow.log_metrics(metrics=metrics,run_id=run_child.info.run_id)

In [21]:
#mlflow.set_tags